Building a recommendation system that predicts the binary outcome of whether the user will like the movie or not, by leveraging Boltzmann Machine. 

Datasets are taken from https://grouplens.org/datasets/movielens/


## Data Preprocessing

In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
# Importing the datasets
movies = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [3]:
movies.head(15)

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [4]:
movies.shape

(3883, 3)

In [5]:
# Importing the datasets
users = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [6]:
users.shape

(6040, 5)

In [7]:
users.head(10)

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


Description of Data:  
0 --> User ID  
1 --> Gender  
2 --> Age  
3 --> Occupation  
4 --> Zip Code  

In [8]:
# Importing the datasets
ratings = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings.shape

(1000209, 4)

In [9]:
ratings.head(10)

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


Descrption of the Data:  
0 --> Corresponds to the user ID   
1 --> Corresponds to the movie ID  
2 --> Ratings from 1-5  
3 --> Timestamps

In [10]:
# Preparing the training set and the test set
training_set = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ml-100k/u1.base', delimiter = '\t')

In [11]:
training_set.head()

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561


In [12]:
training_set.shape

(79999, 4)

In [13]:
training_set = np.array(training_set, dtype = 'int')

In [14]:
test_set = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [15]:
# Getting the total no.of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_users

943

In [16]:
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
nb_movies

1682

In [17]:
# Converting he data into an array with users in lines and movies in columns
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1][data[:, 0] == id_users] 
    id_ratings = data[:, 2][data[:, 0] == id_users] 
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))

  return new_data

training_set = convert(training_set)

In [18]:
test_set = convert(test_set)

In [19]:
print(pd.DataFrame(training_set).head())

   0     1     2     3     4     5     ...  1676  1677  1678  1679  1680  1681
0   0.0   3.0   4.0   3.0   3.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
1   4.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
2   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
3   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
4   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0

[5 rows x 1682 columns]


In [20]:
print(pd.DataFrame(test_set).head())

   0     1     2     3     4     5     ...  1676  1677  1678  1679  1680  1681
0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
1   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
2   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
3   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
4   4.0   3.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0

[5 rows x 1682 columns]


In [21]:
# Converting the data into torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [22]:
training_set.shape

torch.Size([943, 1682])

In [23]:
# Converting the ratings into binary ratings 1 (liked) or 0 (didn't liked)
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

In [24]:
# Converting the ratings into binary ratings 1 (liked) or 0 (didn't liked)
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

## Building the Neural network

In [25]:
# Creating the architectuer of the NN
class RBM():
  def __init__(self, nv, nh):
    self.W = torch.randn(nh, nv) # initializes a tensor od size nh, nv according to normal distribution
    self.a = torch.randn(1, nh) # tensor of nh elements. Visible bias
    self.b = torch.randn(1, nv) # tensor of nv elements. Hidden bias
  
  # Sampling the hidden nodes according to P(h|v) (sigmoid activation function)
  def sample_h(self, x):
    wx = torch.mm(x, self.W.t())
    activation = wx + self.a.expand_as(wx)
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)
  
  # Sampling the visible nodes according to P(v|h) (sigmoid activation function)
  def sample_v(self, y):
    wy = torch.mm(y, self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)

  # k-step Contrastive Divergence
  def train(self, v0, vk, ph0, phk):
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)

nv = len(training_set[0]) # no.of visible nodes
nh = 100 # no.of hidden nodes
batch_size = 100 
rbm = RBM(nv, nh)

## Training the Neural Network

In [29]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
  # loss function
  train_loss = 0
  s = 0.
  for id_user in range(0, nb_users - batch_size, batch_size):
    vk = training_set[id_user:id_user+batch_size]
    v0 = training_set[id_user:id_user+batch_size]
    ph0,_ = rbm.sample_h(v0)
    for k in range(10):
      _,hk = rbm.sample_h(vk)
      _,vk = rbm.sample_v(hk)
      vk[v0 < 0] = v0[v0 < 0]
    phk,_ = rbm.sample_h(vk)
    rbm.train(v0, vk, ph0, phk)
    train_loss += np.sqrt(torch.mean((v0[v0 >= 0] - vk[v0 >= 0])**2)) #RMSE
    s += 1.
  print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.4994)
epoch: 2 loss: tensor(0.4955)
epoch: 3 loss: tensor(0.4974)
epoch: 4 loss: tensor(0.4967)
epoch: 5 loss: tensor(0.4958)
epoch: 6 loss: tensor(0.4916)
epoch: 7 loss: tensor(0.4969)
epoch: 8 loss: tensor(0.4979)
epoch: 9 loss: tensor(0.4920)
epoch: 10 loss: tensor(0.4964)


## Testing the Neural Network

In [30]:
# loss function
test_loss = 0
s = 0.
for id_user in range(nb_users):
  v = training_set[id_user:id_user+1]
  vt = test_set[id_user:id_user+1]
  if len(vt[vt>=0]) > 0:
    _,h = rbm.sample_h(v)
    _,v = rbm.sample_v(h)
    test_loss += np.sqrt(torch.mean((vt[vt >= 0] - v[vt >= 0])**2))
    s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.4834)
